# Pydantic (Row-Level Validation + Slack Notification)

## Chapter 1: Preparation and Libraries

In [2]:
import pandas as pd
import requests
import json
from datetime import datetime
from typing import Optional
from pydantic import BaseModel, Field, field_validator, ConfigDict, ValidationError


MY_WEBHOOK_URL = "WEBHOOK_URL"

# Upload the data
df = pd.read_csv("/content/drive/MyDrive/Colab Data/Amazon Sale Report.csv", low_memory=False)

# Standardise column names (ship-country -> ship_country)
df.columns = [c.replace(' ', '_').replace('-', '_').lower() for c in df.columns]

## Chapter 2: The Pydantic Model

In [3]:
class AmazonOrderRow(BaseModel):
    # V2 configuration: Ignore extra columns and process numbers flexibly
    model_config = ConfigDict(extra='ignore', coerce_numbers_to_str=True)

    order_id: str
    qty: int = Field(ge=0)
    amount: Optional[float] = 0.0
    currency: Optional[str] = "INR"
    ship_country: Optional[str] = "IN"
    date: Optional[datetime] = None

    @field_validator('date', mode='before')
    @classmethod
    def parse_date(cls, v):
        if pd.isna(v) or v == "": return None
        try:
            return pd.to_datetime(v)
        except:
            return None

    @field_validator('amount', mode='before')
    @classmethod
    def handle_nan_amount(cls, v):
        if pd.isna(v): return 0.0
        return v

In [4]:
# Inflexible Pydantic Model
"""
class AmazonOrderRow(BaseModel):
    # Pydantic v2 configuration
    model_config = ConfigDict(extra='ignore', coerce_numbers_to_str=True)

    order_id: str
    qty: int = Field(ge=0)
    amount: Optional[float] = Field(default=0.0, ge=0)
    currency: Optional[str] = "INR"
    ship_country: Optional[str] = "IN"
    date: Optional[datetime] = None

    @field_validator('date', mode='before')
    @classmethod
    def parse_date(cls, v):
        if pd.isna(v) or v == "": return None
        try:
            return datetime.strptime(str(v), '%m-%d-%y')
        except ValueError:
            return pd.to_datetime(v)

    @field_validator('amount', mode='before')
    @classmethod
    def handle_nan_amount(cls, v):
        # Normalise NaN values to 0.0
        if pd.isna(v): return 0.0
        return v
"""

'\nclass AmazonOrderRow(BaseModel):\n    # Pydantic v2 configuration\n    model_config = ConfigDict(extra=\'ignore\', coerce_numbers_to_str=True)\n\n    order_id: str\n    qty: int = Field(ge=0)\n    amount: Optional[float] = Field(default=0.0, ge=0) \n    currency: Optional[str] = "INR"\n    ship_country: Optional[str] = "IN"\n    date: Optional[datetime] = None\n\n    @field_validator(\'date\', mode=\'before\')\n    @classmethod\n    def parse_date(cls, v):\n        if pd.isna(v) or v == "": return None\n        try:\n            return datetime.strptime(str(v), \'%m-%d-%y\')\n        except ValueError:\n            return pd.to_datetime(v)\n\n    @field_validator(\'amount\', mode=\'before\')\n    @classmethod\n    def handle_nan_amount(cls, v):\n        # Normalise NaN values to 0.0\n        if pd.isna(v): return 0.0\n        return v\n'

## Chapter 3: The Verification Cycle

In [5]:
valid_data = []
invalid_data = []

# Line-by-line verification
records = df.to_dict('records')
for record in records:
    try:
        validated_row = AmazonOrderRow(**record)
        valid_data.append(validated_row.model_dump())
    except Exception as e:
        record['validation_error'] = str(e)
        invalid_data.append(record)

# Save the files
pd.DataFrame(valid_data).to_csv("valid_rows.csv", index=False)
pd.DataFrame(invalid_data).to_csv("invalid_rows.csv", index=False)

print(f"Successful: {len(valid_data)} | Unsuccessful: {len(invalid_data)}")

Successful: 128975 | Unsuccessful: 0


## Chapter 4: Slack Notification (Including Test Mode)

In [6]:
def send_pydantic_alert(invalid_count, webhook_url):
    if invalid_count > 0:
        summary = (
            f"🚨 *Pydantic Row-Level Validation Alert!*\n"
            f"❌ Found *{invalid_count}* invalid rows.\n"
            f"📂 Check `invalid_rows.csv` for details."
        )
    else:
        summary = f"✅ *Pydantic Validation Complete:* All {len(valid_data)} rows are valid!"

    payload = {"text": summary}
    response = requests.post(webhook_url, json=payload)

    if response.status_code == 200:
        print("Slack notification successfully sent!")
    else:
        print(f"Slack error: {response.status_code}, {response.text}")

# Call the function
send_pydantic_alert(len(invalid_data), MY_WEBHOOK_URL)

Slack notification successfully sent!
